8. Reinf learning. Exercise: Use policy gradients to solve OpenAI Gym's LunarLander-v2 environment. You will need to install the Box2D dependencies (%pip install -U gym[box2d]). hands-on-ml book exerices page 623

In [1]:
import numpy as np
import gym
env = gym.make("LunarLander-v2")
obs = env.reset()
obs

array([-0.00629091,  1.4026109 , -0.63721627, -0.36931112,  0.00729639,
        0.14433911,  0.        ,  0.        ], dtype=float32)

In [2]:
import matplotlib.pyplot as plt
# img = env.render(mode="rgb_array")
# env.close()
# plt.imshow(img)
print(env.action_space)
print(env.observation_space)

Discrete(4)
Box(-inf, inf, (8,), float32)


In [3]:
# random policy
# print(np.mean(total_rewards), np.std(total_rewards), np.min(total_rewards), np.max(total_rewards))
# -179.8682271598708 113.57826569586982 -536.6091716341286 11.068357820341546

total_rewards = []
for i_episode in range(200):
    rewards = 0
    obs = env.reset()
    for t in range(200):
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        rewards += reward
        if done:
            break
    total_rewards.append(rewards)
print(np.mean(total_rewards), np.std(total_rewards), np.min(total_rewards), np.max(total_rewards))

-176.62070072053183 105.6778123045778 -556.5619316341206 48.6481593929104


In [4]:
# model

import tensorflow as tf
from tensorflow import keras

print(env.reset()[np.newaxis, :])
print(env.action_space.n)
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(env.observation_space.shape[0],)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(12, activation='relu'))
model.add(keras.layers.Dense(env.action_space.n, activation='softmax'))

model(env.reset()[np.newaxis, :])
print(model.trainable_variables)

/Users/mkhokhlush/github/ml-experiments/.venv/lib/python3.8/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/Users/mkhokhlush/github/ml-experiments/.venv/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/Users/mkhokhlush/github/ml-experiments/.venv/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/Users/mkhokhlush/github/ml-experiments/.venv/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resam

[[-0.00632381  1.4050009  -0.64054596 -0.26309383  0.00733448  0.14509319
   0.          0.        ]]
4
[<tf.Variable 'dense/kernel:0' shape=(8, 32) dtype=float32, numpy=
array([[-0.16506653, -0.10286504,  0.3485502 ,  0.21087742, -0.09666863,
         0.30440378,  0.20872843, -0.01611751, -0.29571205,  0.18467975,
         0.37976182,  0.3422675 ,  0.29855305, -0.23479927,  0.38491297,
         0.27555   ,  0.03468424, -0.33295113, -0.37943953,  0.09510672,
         0.2147519 , -0.33260486,  0.3708518 , -0.2820009 ,  0.25632656,
        -0.08792716,  0.07044172, -0.33130547,  0.19710422, -0.32593948,
        -0.32950035, -0.19407587],
       [-0.24595226, -0.17293982, -0.14953186,  0.02069542,  0.00083077,
         0.24914914,  0.3541183 , -0.18796727, -0.13476849, -0.24621682,
        -0.03331041, -0.14625455, -0.33777365,  0.08114955, -0.30508697,
         0.15389633, -0.29010826,  0.27378392, -0.21701051,  0.2444629 ,
        -0.08513954,  0.2243836 , -0.19367751,  0.1701566 ,  0.0

In [17]:
# training
# approach from the book
# logits = tf.math.log(probas + keras.backend.epsilon())
# action = tf.random.categorical(logits, num_samples=1)
def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        action_proba = model(obs[np.newaxis, :])[0]
        rand = tf.random.uniform([env.action_space.n]) # choose random action according to actions' probability
        actions = action_proba - rand
        max_action_value = tf.reduce_max(actions)
        action = int(tf.argmax(actions))
        y_target = tf.cast(actions == max_action_value, tf.float32) # right action is equal to 1 (action above) and 0s (not action) 
        # print(action_proba, y_target)
        loss = tf.reduce_mean(loss_fn(y_target, action_proba))
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, _ = env.step(action)
    return obs, reward, done, grads
play_one_step(env, env.reset(), model, keras.losses.categorical_crossentropy)

(array([ 0.00834646,  1.3870767 ,  0.427131  , -0.50835776, -0.0089152 ,
        -0.08303298,  0.        ,  0.        ], dtype=float32),
 3.1594633717526053,
 False,
 [<tf.Tensor: shape=(8, 32), dtype=float32, numpy=
  array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
           5.4464461e-08, -1.7146746e-08, -2.4070822e-08,  4.7074707e-08,
           0.0000000e+00,  4.7438562e-08,  0.0000000e+00,  0.0000000e+00,
           0.0000000e+00, -3.1500615e-08,  0.0000000e+00,  2.7008006e-08,
           0.0000000e+00, -3.4097862e-08,  5.1876242e-08,  0.0000000e+00,
           5.7299456e-08,  7.7531555e-09,  0.0000000e+00, -2.8408389e-08,
           9.0709316e-09, -2.1691479e-08, -8.7921661e-09,  0.0000000e+00,
          -4.2007219e-08,  0.0000000e+00,  0.0000000e+00, -2.1507704e-08],
         [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
           1.8500690e-05, -5.8244705e-06, -8.1764665e-06,  1.5990512e-05,
           0.0000000e+00,  1.6114107e-05, 

In [6]:
def play_multiple_episodes(env, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = []
    all_grads = []
    for episode in range(n_episodes):
        current_rewards = []
        current_grads = []
        obs = env.reset()
        for step in range(n_max_steps):
            obs, reward, done, grads = play_one_step(env, obs, model, loss_fn)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads

def discount_rewards(rewards, discount_factor=0.97):
    discounted = np.array(rewards)
    for n_reward in range(len(discounted) - 2, -1, -1):
        discounted[n_reward] += discount_factor * discounted[n_reward + 1]
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_factor=0.97):
    all_discounted_rewards = [discount_rewards(rewards, discount_factor) for rewards in all_rewards]
    flat_all_discounted_rewards = np.concatenate(all_discounted_rewards)
    flat_mean = np.mean(flat_all_discounted_rewards)
    flat_std = np.std(flat_all_discounted_rewards)
    return [(discounted_rewards - flat_mean) / flat_std for discounted_rewards in all_discounted_rewards]

print(discount_rewards([10, 0, -50], 0.8)) # -22 -40 -50
print(discount_and_normalize_rewards([[10, 0, -50], [10, 20]], 0.8)) # -0.28 .... 1.26, 1.07

[-22 -40 -50]
[array([-0.28435071, -0.86597718, -1.18910299]), array([1.26665318, 1.0727777 ])]


In [7]:
# training algorithm: play several episodes without applying grads, then apply mean grad according to a reward
from tqdm import tqdm

n_iterations = 300
n_episodes_per_update = 12
n_max_steps = 200
discount_factor = 0.95

optimizer = keras.optimizers.Adam(0.01)
loss_fn = keras.losses.categorical_crossentropy

def train():
    for n_iteration in tqdm(range(n_iterations)):
        all_rewards, all_grads = play_multiple_episodes(env, n_episodes_per_update, n_max_steps, model, loss_fn)
        all_final_rewards = discount_and_normalize_rewards(all_rewards, discount_factor)
        
        mean_grads = []
        for trainable_var_index in range(len(model.trainable_variables)):
            weighted_grads = []
            for episode, final_rewards in enumerate(all_final_rewards):
                for step, final_reward in enumerate(final_rewards):
                    grad = all_grads[episode][step][trainable_var_index]
                    weighted_grads.append(grad * final_reward) # the more reward the more grad is important
            mean_grad = tf.reduce_mean(weighted_grads, axis=0) # reduce mean to get a mean grad between all episodes and steps
            mean_grads.append(mean_grad)
        optimizer.apply_gradients(zip(mean_grads, model.trainable_variables))    
train()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [41:16<00:00,  8.25s/it]


In [8]:
# play with the trained model

total_rewards = []
for i_episode in tqdm(range(200)):
    rewards = 0
    obs = env.reset()
    for t in range(200):
        action_proba = model(obs[np.newaxis, :])[0]
        action = int(tf.argmax(action_proba))
        obs, reward, done, info = env.step(action)
        rewards += reward
        if done:
            break
    total_rewards.append(rewards)
# 29.660429310869976 24.329520575340702 -25.517524436082713 98.75970244131753
print(np.mean(total_rewards), np.std(total_rewards), np.min(total_rewards), np.max(total_rewards))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:48<00:00,  4.15it/s]

29.660429310869976 24.329520575340702 -25.517524436082713 98.75970244131753
